In [9]:
%pip install "qwen-agent[code_interpreter,python_executor]"

# code_interpreter: Code interpreter
# python_executor: Python executor

Note: you may need to restart the kernel to use updated packages.


## Using Ollama for Qwen Model

In this example, we'll be using Ollama to pull and run the Qwen2.5 7B model. Ollama provides an easy way to run large language models locally.


In [5]:
# Pull qwen model using ollama
!ollama pull qwen2.5:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         
pulling eb4402837c78... 100% ▕████████████████▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕████████████████▏  11 KB                         
pulling 2f15b3218f05... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [7]:
# Config llm model

llm_cfg = {
    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    'model': 'qwen2.5:7b',
    'model_server': 'http://localhost:11434/v1/',  # base_url, also known as api_base
    'api_key': 'EMPTY',

    # (Optional) LLM hyperparameters for generation:
    'generate_cfg': {
        'top_p': 0.8
    }
}

In [10]:
system_prompt = """
You are a python expert. You should follow the instructions below to answer user's questions.
- Think about what is the expected output for the program user wants to create.
- Then write a Python code to achieve this goal.
- Then run the code to check if it works.
- If the code result is not what we excepted, you should rewrite the code.
- If the code result is what we excepted, you should write the code result to the answer.
"""

tools = ['code_interpreter', 'python_executor']

In [ ]:
# Create the qwen agent

from qwen_agent.agents import Assistant
bot = Assistant.create(
    llm=llm_cfg,
    function_list=tools,
    system_message=system_prompt
)